In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from fullfrlm import flow_refueling_location_model
from generate_network import generate_network
from flow_computation import flow_computation
from random_vessel_generator import random_vessel_generator
from first_stage_frlm import first_stage_frlm
from second_stage_frlm import second_stage_frlm
from create_input_data_abm import create_input_data_abm
from visualize_placement import visualize_placement
from get_abm_data import get_vessel_data_batch, get_cs_data_batch
import pickle
from additional_intersections_new import additional_intersections
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import time
import pulp as pl

In [2]:
solver_list = pl.listSolvers()

In [4]:
solver_list = pl.listSolvers(onlyAvailable=True)

In [5]:
solver_list

['PULP_CBC_CMD']

In [3]:
G = pickle.load(open('data/network_cleaned_final.p', 'rb'))
df_h = pickle.load(open("data/revised_cleaning_results/harbour_data_100.p", "rb"))
df_ivs = pickle.load(open("data/revised_cleaning_results/ivs_exploded_100.p", "rb"))
path_lengths = pickle.load(open("data/revised_cleaning_results/path_lengths_ship_specific_routes.p", "rb"))
paths = pickle.load(open('data/final_paths.p', "rb"))

In [11]:
df_h

,harbour_code,lat,lon,harbour_node
0,NLABL,51.861514,4.657490,8866686
1,NLALK,52.633333,4.750000,8861625
2,NLALM,52.350000,6.666667,8866512
3,NLAER,52.366667,5.216667,8864005
4,NLAPN,52.133333,4.666667,8863360
...,...,...,...,...
95,NLWMO,52.500000,4.816667,8866429
96,NLZAA,52.442040,4.829199,8866429
97,NLZWS,52.640869,6.070316,8860582
98,NLZWI,51.810598,4.627272,8867130


In [44]:
a = df_ivs.groupby(['origin','destination', 'route_v']).sum().reset_index().sort_values('trip_count', ascending=False)
a = a.loc[((a.origin=='NLRTM') & (a.destination=='NLAMS'))|((a.origin == 'NLRTM') & (a.destination == 'NLMOE'))|((a.origin == 'NLRTM') & (a.destination == 'NLKAM'))]
a = a.drop(columns='hour')
m2 = (a !=0).any()
a = a[m2.index[m2].tolist()]
a.to_excel('data/excel_files/splitting.xlsx', sheet_name='splitted_data')

In [45]:
a

,origin,destination,route_v,trip_count,M12,M8,M10,BIIa-1,M9,C3b,...,BII-1,BII-2b,M4,B03,C4,B04,C2l,BII-2L,B02,C2b
177,NLRTM,NLAMS,1,6103,0,3510,0,102,1316,0,...,97,0,0,0,0,0,7,0,0,0
178,NLRTM,NLAMS,2,3513,2076,0,427,0,0,11,...,0,132,0,0,5,0,0,39,0,0
203,NLRTM,NLMOE,0,2174,0,1584,0,2,144,0,...,0,0,1,0,0,0,0,0,0,0
197,NLRTM,NLKAM,2,931,6,430,150,13,224,0,...,2,3,0,0,0,0,1,5,0,1
204,NLRTM,NLMOE,1,511,70,0,56,0,0,324,...,0,1,0,0,2,0,0,1,0,0
176,NLRTM,NLAMS,0,316,0,0,0,0,0,0,...,0,0,65,19,0,5,0,0,4,0
196,NLRTM,NLKAM,1,62,0,0,0,0,0,0,...,0,0,13,0,0,0,0,0,0,0
195,NLRTM,NLKAM,0,43,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
path_lengths[('NLRTM', 'NLAMS', 0)]

108574.45594726905

In [50]:
path_lengths[('NLRTM', 'NLAMS', 1)]

144456.3149692275

In [51]:
path_lengths[('NLRTM', 'NLAMS', 2)]

144909.0652576073

In [54]:
routes_types_paths = pickle.load(open("data/revised_cleaning_results/users_ship_specific_routes.p", "rb" ))

In [56]:
routes_types_paths

{('NLABL',
  'NLRTM'): {0: ['I',
   'II',
   'III',
   'IV',
   'V_A',
   'V_B',
   'VI_A',
   'VI_B',
   'VI_C']},
 ('NLAER',
  'NLLEY'): {0: ['I',
   'II',
   'III',
   'IV',
   'V_A',
   'V_B',
   'VI_A',
   'VI_B',
   'VI_C']},
 ('NLAER',
  'NLNIE'): {0: ['I',
   'II',
   'III',
   'IV',
   'V_A',
   'V_B',
   'VI_A',
   'VI_B',
   'VI_C']},
 ('NLAER',
  'NLRTM'): {0: ['I',
   'II',
   'III',
   'IV',
   'V_A',
   'V_B',
   'VI_A',
   'VI_B',
   'VI_C']},
 ('NLAER',
  'NLSWI'): {0: ['I',
   'II',
   'III',
   'IV',
   'V_A',
   'V_B',
   'VI_A',
   'VI_B',
   'VI_C']},
 ('NLALK',
  'NLIJM'): {0: ['I',
   'II',
   'III',
   'IV',
   'V_A',
   'V_B',
   'VI_A',
   'VI_B',
   'VI_C']},
 ('NLALM',
  'NLAMS'): {0: ['I',
   'II',
   'III',
   'IV',
   'V_A',
   'V_B',
   'VI_A',
   'VI_B',
   'VI_C']},
 ('NLALM',
  'NLHGL'): {0: ['I',
   'II',
   'III',
   'IV',
   'V_A',
   'V_B',
   'VI_A',
   'VI_B',
   'VI_C']},
 ('NLAMS',
  'NLAER'): {0: ['I',
   'II',
   'III',
   'IV',
   'V_A',
 

In [57]:
routes_types_paths[('NLRTM', 'NLAMS')]

{0: ('I', 'II', 'III'), 1: ('V_A', 'IV'), 2: ('VI_B', 'VI_C', 'V_B', 'VI_A')}

In [46]:
b = df_ivs.groupby(['origin','destination']).sum().reset_index().sort_values('trip_count', ascending=False)
b = b.loc[((b.origin=='NLRTM') & (b.destination=='NLAMS'))|((b.origin == 'NLRTM') & (b.destination == 'NLMOE'))|((b.origin == 'NLRTM') & (b.destination == 'NLKAM'))]
b = b[m2.index[m2].tolist()]
b = b.drop(columns=['route_v'])

with pd.ExcelWriter('data/excel_files/splitting.xlsx',
                    mode='a') as writer:
    b.to_excel(writer, sheet_name='non_splitted_data')

In [47]:
b

,origin,destination,trip_count,M12,M8,M10,BIIa-1,M9,C3b,BII-4,...,BII-1,BII-2b,M4,B03,C4,B04,C2l,BII-2L,B02,C2b
120,NLRTM,NLAMS,9932,2076,3510,427,102,1316,11,5,...,97,132,65,19,5,5,7,39,4,0
136,NLRTM,NLMOE,2685,70,1584,56,2,144,324,0,...,0,1,1,0,2,0,0,1,0,0
132,NLRTM,NLKAM,1036,6,430,150,13,224,0,0,...,2,3,13,0,0,0,1,5,0,1


In [4]:
# pick seeds
import numpy as np
pick_from = np.linspace(0, 1000000, 1000001)
seeds = np.random.choice(a=pick_from, size=100, replace=False)
seeds = list(seeds)
seeds = [round(i) for i in seeds]

In [5]:
seeds

[192851,
 371193,
 156320,
 42774,
 564023,
 386760,
 780624,
 426344,
 874076,
 98930,
 277793,
 508119,
 505579,
 374666,
 11992,
 314837,
 661554,
 68156,
 405904,
 501475,
 123614,
 636170,
 284327,
 589233,
 336965,
 10364,
 145042,
 565816,
 233029,
 945701,
 349380,
 711944,
 206118,
 47019,
 452477,
 384818,
 683699,
 500264,
 555018,
 590007,
 602409,
 635303,
 337621,
 327822,
 482862,
 458723,
 275710,
 115423,
 19682,
 387205,
 374496,
 909574,
 560151,
 307041,
 825225,
 406556,
 260588,
 498575,
 54380,
 864866,
 270439,
 568011,
 288159,
 620269,
 69488,
 848783,
 423745,
 230617,
 354074,
 865052,
 619527,
 963402,
 611107,
 812032,
 47616,
 910883,
 366632,
 8970,
 747140,
 482276,
 41869,
 46033,
 22259,
 303654,
 316429,
 508825,
 658250,
 675389,
 130248,
 676693,
 234640,
 81926,
 342218,
 968753,
 563040,
 587633,
 814902,
 196538,
 446545,
 563796]

In [6]:
seeds = [259572, 7542, 864199, 11540, 188813, 411862, 546078, 420812, 304196, 814030]

In [7]:
%%time
total_flow, fraction_captured_total, serviceable_fraction, served_fraction, optimal_facilities, \
    non_zero_flows, routes_supported, paths, G, df_abm, df_random_out, max_flow = flow_refueling_location_model(1, seeds[0], 150000, 15000, 175, 100, 600, 2, 24, False, False, False, True)

KeyboardInterrupt: 

In [8]:
serviceable_fraction

NameError: name 'serviceable_fraction' is not defined

In [ ]:
non_zero_flows

In [ ]:
df_h = pickle.load(open("data/revised_cleaning_results/harbour_data_100.p", "rb"))
facilities = [i for i in optimal_facilities.keys() if optimal_facilities[i] > 0]

extra_nodes_used = list(set(facilities)-set(list(df_h.harbour_node)))

In [ ]:
print(total_flow, fraction_captured_total, serviceable_fraction, served_fraction, routes_supported)

# Complete batch run first before continuing
Run file batchrun.py in ABM/ownwork

## Rerun using feedback from ABM

In [ ]:
vessel_data_abm = get_vessel_data_batch()

In [ ]:
station_data_abm = get_cs_data_batch()

In [ ]:
station_data_abm

In [ ]:
top5_least_used = list(station_data_abm.sort_values(by='occupation').head(5).index)

In [ ]:
top5_least_used

In [ ]:
%%time
total_flow, fraction_captured_total, serveable_fraction, served_fraction, optimal_facilities, \
non_zero_flows, routes_supported, paths, G, df_abm, df_random_out = flow_refueling_location_model(load, 70000, 40, 1000, 2, df_random, False, False, True, top5_least_used)

In [ ]:
facilities = [i for i in optimal_facilities.keys() if optimal_facilities[i] > 0]
extra_nodes_used = list(set(facilities) - set(list(df_h.harbour_node)))
extra_nodes_used

In [ ]:

print(total_flow, fraction_captured_total, serveable_fraction, served_fraction, routes_supported)

## Now simulate again

## Stepwise run of all components of frlm

### Load necessary data

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from fullfrlm import flow_refueling_location_model
from generate_network import generate_network
from flow_computation import flow_computation
from random_vessel_generator import random_vessel_generator
from first_stage_frlm import first_stage_frlm
from second_stage_frlm import second_stage_frlm
from create_input_data_abm import create_input_data_abm
from visualize_placement import visualize_placement
from get_abm_data import get_vessel_data_batch, get_cs_data_batch
import pickle
from additional_intersections_new import additional_intersections
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import time

In [2]:
G = pickle.load(open('data/network_cleaned_final.p', 'rb'))
df_h = pickle.load(open("data/revised_cleaning_results/harbour_data_100.p", "rb"))
df_ivs = pickle.load(open("data/revised_cleaning_results/ivs_exploded_100.p", "rb"))
path_lengths = pickle.load(open("data/revised_cleaning_results/path_lengths_ship_specific_routes.p", "rb"))
paths = pickle.load(open('data/final_paths.p', "rb"))

In [3]:
df_ivs

,origin,destination,trip_count,hour,M12,M8,BII-6b,M10,BIIa-1,M9,...,C4,B04,C2l,BII-2L,B02,C1b,C2b,B01,C1l,route_v
1,NLRTM,NLAMS,171,0,0,109,0,0,2,45,...,0,0,0,0,0,0,0,0,0,1
2,NLRTM,NLAMS,152,0,91,0,0,18,0,0,...,0,0,0,1,0,0,0,0,0,2
4,NLRTM,NLAMS,178,1,0,110,0,0,3,55,...,0,0,0,0,0,0,0,0,0,1
5,NLRTM,NLAMS,124,1,78,0,0,17,0,0,...,0,0,0,0,0,0,0,0,0,2
7,NLRTM,NLAMS,179,2,0,122,0,0,3,46,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7746,NLVLI,NLVLA,1,21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7747,NLVLI,NLVLA,3,21,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
7749,NLVLI,NLVLA,5,22,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7750,NLVLI,NLVLA,1,23,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



## Compute path flows

In [4]:
%%time
flows = flow_computation(df_ivs, 1000000, path_lengths)

CPU times: total: 46.9 ms
Wall time: 227 ms


In [5]:
sum(flows.values())

2941841.190476222

## Generate network

In [6]:
%%time
# G = pickle.load(open('data/network_cleaned_final.p', 'rb'))
# paths = pickle.load(open("data/final_paths.p", "rb"))
G, paths, inserted = generate_network(G, paths, 5)
inserted += additional_intersections(G, 5)

There were 5 nodes added, the longest remaining link is now: 48533.206009320114
['8864435', '8868065', '8867046', '8863114', '8861611'] intersections added
CPU times: total: 938 ms
Wall time: 4.07 s


In [11]:
%%time
# range r in meters
df_b, df_g, df_eq_fq, feasible_combinations = first_stage_frlm(150000, G, OD=flows, paths=paths, path_lengths=path_lengths, df_h=df_h, additional_nodes=inserted)

CPU times: total: 562 ms
Wall time: 2.43 s


In [12]:
df_eq_fq['f_q'].sum()
# check

2941841.190476221

## Run second stage FRLM

In [13]:
%%time
optimal_facilities, optimal_flows, non_zero_flows, supported_flow, routes_supported = second_stage_frlm(10, 5, 2000, 24, df_g, df_b, df_eq_fq)

CFRLM:
MAXIMIZE
2703.5759068704288*Flow_captured_(('NLABL',_'NLRTM',_0),_('8863306',)) + 2703.5759068704288*Flow_captured_(('NLABL',_'NLRTM',_0),_('8866686',)) + 2212.1197716431*Flow_captured_(('NLAER',_'NLLEY',_0),_('8863709',)) + 2212.1197716431*Flow_captured_(('NLAER',_'NLLEY',_0),_('8864005',)) + 11277.026528683964*Flow_captured_(('NLAER',_'NLNIE',_0),_('8864005',_'8863360')) + 11277.026528683964*Flow_captured_(('NLAER',_'NLNIE',_0),_('8864005',_'8867538')) + 11277.026528683964*Flow_captured_(('NLAER',_'NLNIE',_0),_('8867046',)) + 29584.428391712187*Flow_captured_(('NLAER',_'NLRTM',_0),_('8863306',_'8864005')) + 29584.428391712187*Flow_captured_(('NLAER',_'NLRTM',_0),_('8863306',_'8867046')) + 29584.428391712187*Flow_captured_(('NLAER',_'NLRTM',_0),_('8863360',_'8867046')) + 29584.428391712187*Flow_captured_(('NLAER',_'NLRTM',_0),_('8864005',_'8863360')) + 29584.428391712187*Flow_captured_(('NLAER',_'NLRTM',_0),_('8864005',_'8867538')) + 29584.428391712187*Flow_captured_(('NLAER',_

In [14]:
sum(optimal_facilities.values())

10.0

In [15]:
for route, combi_flow in non_zero_flows.items():
    print(sum(combi_flow['flows']))

1.0
1.0
1.0
1.0
0.62720039
1.0
1.0
0.011521488
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.95852191
1.0
1.0
0.20359958
0.16527803
1.0
0.5441609
1.0
1.0
0.10789298
1.0
1.0
1.0
1.0
1.0
1.0
0.65428949
1.0
1.0
1.0


## Create input data ABM

In [ ]:
%%time
df_abm = create_input_data_abm(G, paths, non_zero_flows, optimal_facilities)

In [ ]:
df_abm

## Visualize placement

In [ ]:
visualize_placement(G,flows, optimal_facilities, non_zero_flows, df_h, paths, True)